In [ ]:
# Installing required libraries
!pip install textblob agno duckduckgo-search
!python -m textblob.downloade_corpora
!pip install groq
!pip install yfinance
!pip install newspaper3k


/usr/bin/python3: No module named textblob.downloade_corpora


In [ ]:
!pip install ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [ ]:
# importing agent componant and tools
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools
from agno.team import Team

In [ ]:
import os
from getpass import getpass

# Prompt user for Groq API key without showing it on screen
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")



Enter your Groq API Key: ··········


In [ ]:
# Defining a Sentiment Analysis Tool using TextBlob
from agno.tools import tool
from textblob import TextBlob

@tool("TextBlob Sentiment Tool")
def analyze_sentiment(text: str) -> str:
    """Analyzes sentiment using TextBlob"""
    blob = TextBlob(text)
    sentiment = blob.sentiment
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity
    interpretation = (
        "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"
    )
    return (
        f"Sentiment Analysis:\n"
        f"- Polarity: {polarity:.2f}\n"
        f"- Subjectivity: {subjectivity:.2f}\n"
        f"- Interpretation: {interpretation}"
    )

In [ ]:
# creating sentiment analysis agents
sentiment_agent = Agent(
    name="Sentiment Analyst",
    role="Analyze sentiment of financial news headlines and identify overall tone.",
    model = Groq(id="llama-3.3-70b-versatile"),
    tools=[analyze_sentiment],
    instructions="Analyze the sentiment of financial news to know the market perception about the company.",
    markdown=True
)


In [ ]:
# creating web search agent
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model = Groq(id="llama-3.3-70b-versatile"),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    markdown=True
)

In [ ]:
# creating finace data agent
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model = Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools],
    instructions="Use tables to display data",
    markdown=True
)


In [ ]:
agent_team = Team(
    name="Financial Analysis Team",
    model = Groq(id="llama-3.3-70b-versatile"),
    members=[web_agent, finance_agent, sentiment_agent],
    instructions=[
        "Collect latest news related to company.",
        "Fetch current stock price and basic financial info.",
        "Analyze sentiment of each headline.",
        "Then summarize findings as sections: News, Data Insights, Sentiment Summary."
    ],
    markdown=True
)

In [ ]:
agent_team.print_response("Please give top 5 news of Microsoft and analyze the sentiment along with scores. Does the sentiment have impact on stock price?", stream=True)

Output()

WARNING  Could not add tool <class 'agno.tools.yfinance.YFinanceTools'>: Unable to validate <class                 
         'agno.tools.yfinance.YFinanceTools'>: `validate_call` should be applied to functions, not classes (put    
         `@validate_call` on top of `__init__` or `__new__` instead)                                               
                                                                                                                   
         For further information visit https://errors.pydantic.dev/2.11/u/validate-call-type

In [ ]:
agent_team.print_response(
    "Give me a summary of the top 5 latest news about Microsoft and analyze the sentiment with scores. Predict if it may impact the stock price.",
    stream=True
)


Output()